# System Preparation 

This notebook serves as notes for reference to getting setup to develop with SUBER.

- [GPU Preparation](#gpu_prep)
- [PyTorch Installation](#torch_install)
- [Pytorch Examples]()
- [Transformers and Tokenizers]()

TODO -- fix links above.

## Conda or Python Virtual Environments
I switched to ordinary Python virtual environments because Anaconda itself was becoming a chore.  Why would it not simply add the mdodule I wanted?  It would take forever and stall in many cases.  The Python version used for this project is Python 3.10.12.


## <a href="gpu_prep">GPU Preparation</a>

GPU and nvcc (aka cuda) versions should be within the same major version. I've noticed that Ubuntu 22.04 loads on some systems have been way out of **alignment**. Try to get them at the same version.


```
sudo apt-get purge 'nvidia*' 'cuda*'

sudo apt-get install nvidia-driver-535

sudo reboot

wget https://developer.download.nvidia.com/compute/cuda/12.2.0/local_installers/cuda_12.2.0_535.54.03_linux.run

chmod a+x cuda_12.2.0_535.54.03_linux.run
 
sudo ./cuda_12.2.0_535.54.03_linux.run # And follow the prompts

# Edit your .bashrc and put these in. But don't put the hastags in front of them.
# export PATH=/usr/local/cuda-12.2/bin${PATH:+:${PATH}}
#export LD_LIBRARY_PATH=/usr/local/cuda-12.2/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}


# I also had to do this.  If you cannot type nvcc --version then you need to check the permissions.
sudo chmod -R 755 /usr/local/cuda-12.2


```

The results should be something like this:

```
acshell@ip-10-114-92-249:~$ nvidia-smi | grep -i "cuda version" | awk '{print $9}'
12.2
acshell@ip-10-114-92-249:~$ nvcc --version
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Jun_13_19:16:58_PDT_2023
Cuda compilation tools, release 12.2, V12.2.91
Build cuda_12.2.r12.2/compiler.32965470_0

```

## <a href=torch_install>Torch Installation<a/>

You should do this first. If this doesn't work, nothing will. 

PyTorch cuda version should be within a minor version of the cuda drivers and cuda drivers need to align with nvidia drivers.  Try hard to make this happen by paying attention to versions.  

```.bash
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

```


In [1]:

import torch
print(f"Is Cuda available? {torch.cuda.is_available()}.")  # Should return True
print(f"Torch Cuda Version is {torch.version.cuda}.")  # Should return '12.1'

import time

Is Cuda available? False.
Torch Cuda Version is 12.4.


### Torch Examples

Here are some examples showing that it works.

In [2]:
import torch
import time

# Define the size of the tensors
size = 10000

# Create two large random tensors for CPU
tensor1_cpu = torch.randn(size, size)
tensor2_cpu = torch.randn(size, size)

# Perform matrix multiplication on the CPU and time it
start_time = time.time()
result_cpu = torch.matmul(tensor1_cpu, tensor2_cpu)
end_time = time.time()

print(f"Matrix multiplication on CPU took: {end_time - start_time:.4f} seconds")
print(f"Result tensor size on CPU: {result_cpu.size()}")

# Check if CUDA is available and perform the same test on the GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    
    # Create two large random tensors for GPU
    tensor1_gpu = tensor1_cpu.to(device)
    tensor2_gpu = tensor2_cpu.to(device)

    # Perform matrix multiplication on the GPU and time it
    torch.cuda.synchronize()  # Ensure all CUDA operations are finished
    start_time = time.time()
    result_gpu = torch.matmul(tensor1_gpu, tensor2_gpu)
    torch.cuda.synchronize()  # Ensure the GPU has finished the computation
    end_time = time.time()

    print(f"Matrix multiplication on GPU took: {end_time - start_time:.4f} seconds")
    print(f"Result tensor size on GPU: {result_gpu.size()}")
else:
    print("CUDA is not available on this system.")


Matrix multiplication on CPU took: 3.4662 seconds
Result tensor size on CPU: torch.Size([10000, 10000])
CUDA is not available on this system.


## Stable Baselines 3 


## Install Stable Baselines 3

```
pip install stable-baselines3[extra]

```


In [3]:
import os
# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppresses INFO, WARNING, and ERROR messages

import stable_baselines3
import stable_baselines3
print(stable_baselines3.__version__)

2.3.2


### SB3 Example

Note, it takes many iteraitons and the proper algorithm to get good results; this just shows it working.  

In [4]:
import gymnasium as gym
from stable_baselines3 import PPO
import matplotlib.pyplot as plt
from IPython import display
import time

# Create the CartPole-v1 environment with the "rgb_array" render mode
env = gym.make("CartPole-v1", render_mode="rgb_array")

# Create the PPO model (you can replace PPO with other algorithms if you want)
model = PPO("MlpPolicy", env, verbose=1)

# Train the agent for 10,000 steps
model.learn(total_timesteps=10000)

# Test the trained agent and render in the notebook
obs, info = env.reset()

# Set up the plot for dynamic updates
#plt.ion()  # Turn on interactive mode for matplotlib
#fig, ax = plt.subplots()

for _ in range(1000):
    action, _states = model.predict(obs)
    obs, reward, done, truncated, info = env.step(action)


    if done or truncated:
        obs, info = env.reset()

env.close()


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.3     |
|    ep_rew_mean     | 21.3     |
| time/              |          |
|    fps             | 2341     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 27.2         |
|    ep_rew_mean          | 27.2         |
| time/                   |              |
|    fps                  | 2542         |
|    iterations           | 2            |
|    time_elapsed         | 1            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.008960325  |
|    clip_fraction        | 0.1          |
|    clip_range           | 0.2          |
|    en

## Install Transformers and Tokenizers

```
pip install -U transformers tokenizers

```

### Transformer and Tokenizer Example

In [5]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Sample text
text = "Transformers are amazing for NLP tasks."

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt")

# Get the tokenized input IDs
input_ids = inputs["input_ids"]

# Decode the token IDs back to text
decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)

# Print original text, tokenized input, and decoded text
print("Original Text:", text)
print("Tokenized Input IDs:", input_ids)
print("Decoded Text:", decoded_text)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Original Text: Transformers are amazing for NLP tasks.
Tokenized Input IDs: tensor([[  101, 19081,  2024,  6429,  2005, 17953,  2361,  8518,  1012,   102]])
Decoded Text: transformers are amazing for nlp tasks.


## Sentence Transformers

```/bash
pip install sentence-transformers

```

Need this as well.

### Sentence Transformers Example

In [6]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode a list of sentences
sentences = ["Transformers are amazing for NLP tasks.", "Sentence embeddings are useful."]
embeddings = model.encode(sentences)

# Print the sentence embeddings
print(embeddings)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-9.13390219e-02 -2.08391473e-02  3.78195010e-02 -1.00276247e-02
  -2.18986869e-02  6.64145825e-03 -4.42283042e-02  4.97135967e-02
   2.80647688e-02  1.45602077e-02  2.62471680e-02  8.02668929e-02
   4.97582043e-03  8.78880396e-02  4.61269915e-02  3.79769169e-02
   3.22095528e-02  1.52602587e-02 -4.78855744e-02 -8.71268660e-02
   1.09329268e-01  8.22059661e-02  1.47923622e-02 -5.11703007e-02
   5.15240654e-02  6.55859709e-02 -5.36913536e-02 -4.96964194e-02
   3.65458615e-02 -9.47063603e-03 -4.13955376e-02  5.72569109e-02
  -6.83491975e-02  5.84684871e-02 -6.26850054e-02  7.58527368e-02
   1.44350678e-02  1.83785893e-02  1.41707724e-02 -5.39689176e-02
  -3.47521007e-02 -1.90681871e-02  1.80511996e-02 -2.25276481e-02
   4.55970392e-02 -3.47323604e-02 -2.73608956e-02 -1.98490638e-02
  -8.14835308e-04 -2.79271062e-02 -3.95562649e-02 -5.84855452e-02
   5.34983687e-02  1.18243933e-01 -1.34978304e-02  2.81568412e-02
   8.00178573e-03 -5.52488901e-02  1.07748117e-02 -8.26784447e-02
  -7.50691

## Other Stuff

If you are using Jupyter notebook, be sure to install `jupyterlab` and `ipywidgets` with pip.

```.bash




```

## Setup Summary -- 

More notes can be added here but Pytorch, and Stable Baselines 3 are the two main modules.  Extras required from both will come up but should not be a huge issue.  